# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
env = gym.make('Pendulum-v0')

random_seed = 10
env.seed(random_seed)

# from agent import Agent
# agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], 
#               random_seed=random_seed)

/home/arasdar/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[10]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [3]:
env.action_space, env.observation_space

(Box(1,), Box(3,))

In [4]:
action_size = env.action_space.shape[0]
state_size = env.observation_space.shape[0]

In [5]:
action_size, state_size

(1, 3)

In [6]:
env.observation_space.low, env.observation_space.high

(array([-1., -1., -8.], dtype=float32), array([1., 1., 8.], dtype=float32))

In [7]:
env.action_space.low, env.action_space.high

(array([-2.], dtype=float32), array([2.], dtype=float32))

### 3. Instantiate DDPG Agent

In [8]:
from agent2 import Agent

agent = Agent(s_size=state_size, a_size=action_size, random_seed=random_seed)

### 4. Train the Agent with DDPG

In [9]:
from collections import deque
import numpy as np
import time
import torch

n_episodes=1000
# n_steps=1000
n_learn=1
# learn_every=20
R_goal=-100 # TOCHECK
Rs_deque = deque(maxlen=100)
meanRs = [] # plot
Rs = [] # plot
dlosses = []
glosses = []

In [ ]:
for i_episode in range(n_episodes):
    
    # # Reset Env
    # env_info = env.reset(train_mode=train_mode)[brain_name]       # reset the environment    
    # S = env_info.vector_observations                   # get the current state (for each agent)
    s = env.reset() # s: state [-1:1, -1:1, -8:8]
    #s_ = np.zeros_like(s) # s_: predicted state [-1:1, -1:1, -8:8]
    #print(s.shape)

    #Rs = np.zeros(num_agents)                            # initialize the score (for each agent)
    R = 0 # R: total reward
    
    glosses_batch = []
    dlosses_batch = []

    #for i_step in range(n_steps):
    while True:
        s = np.reshape(s, [1, -1])
        
        a = agent.act(s) # a=[-2, 2]
        a *= 2
        #print(a.shape)
        
        # Exploration noise
        # noise = np.random.normal(loc=0, scale=0.1, size=[num_agents, action_size])
        # A += noise
        # A = np.clip(a=A, a_min=-1, a_max=1)

        # env_info = env.step(A)[brain_name]            # send all actions to the environment
        # S2 = env_info.vector_observations          # get next state (for each agent)
        # rewards = env_info.rewards                          # get reward (for each agent)
        # dones = env_info.local_done                         # see if episode finished
        #s2: next_state, r: reward, done/terminal, env: environment
        a = np.reshape(a, [-1])
        s2, r, done, _ = env.step(a)
        #print(s2.shape, r, done)

        #s2_, q_ = agent.env(s, a)
        a = np.reshape(a, [1, -1])
        #s2_, _ = agent.env(s, a)
        # print(s2_)
        # s2_ = np.tanh(s2_) # predicted next state [-1:1, -1:1, -8:8]
        # print(s2_)
        # s2_[2] = s2_[2] * 8
        # print(s2_)

        # for s, a, reward, s2, done in zip(S, A, rewards, S2, dones):
        # agent.step(state, action, reward, next_state, done) # send actions to the agent
        agent.step(s, a, r, s2, done)

        R += r                           # update the score (for each agent)
        s = s2                                # roll over the states to the next state

        # if i_step%learn_every == 0:
        dlosses_Nlearn = []
        for _ in range(n_learn):
            dloss = agent.start_dlearn()
            #print(dloss)
            dlosses_Nlearn.append(dloss)
        dlosses_batch.append(np.mean(dlosses_Nlearn))
            
        #print(len(agent.memory))
        gloss = agent.start_glearn()
        glosses_batch.append(gloss)

        # if np.any(dones):                                   # exit loop if episode finished
        #     break
        if done:
            break

    #meanR = np.mean(Rs)
    #Rs_deque.append(meanR)
    Rs_deque.append(R)
    meanRs.append(np.mean(Rs_deque)) # plot
    Rs.append(R) # plot
    dlosses.append(np.mean(dlosses_batch))
    glosses.append(np.mean(glosses_batch))

    print('\rEpisode {}\tTotal Average Score: {:.2f}\tGloss: {:.2f}, \tDloss: {:.2f}'.format(
           i_episode+1, np.mean(Rs_deque), np.mean(glosses_batch), np.mean(dlosses_batch)))  
    
    if np.mean(Rs_deque) >= R_goal:
        torch.save(agent.g.state_dict(), 'g-pendulum.pth') # pendulum
        torch.save(agent.d.state_dict(), 'd-pendulum.pth') # pendulum
        break

Episode 1	Total Average Score: -883.57	Gloss: 0.00, 	Dloss: 0.00
Episode 2	Total Average Score: -1367.03	Gloss: 0.00, 	Dloss: 0.00
Episode 3	Total Average Score: -1157.02	Gloss: 0.00, 	Dloss: 0.00
Episode 4	Total Average Score: -1088.51	Gloss: 0.00, 	Dloss: 0.00
Episode 5	Total Average Score: -1185.62	Gloss: 0.00, 	Dloss: 0.00
Episode 6	Total Average Score: -1261.77	Gloss: 0.05, 	Dloss: 57.61
Episode 7	Total Average Score: -1296.57	Gloss: 0.10, 	Dloss: 65.27
Episode 8	Total Average Score: -1281.78	Gloss: 0.15, 	Dloss: 65.40
Episode 9	Total Average Score: -1255.09	Gloss: 0.18, 	Dloss: 66.49
Episode 10	Total Average Score: -1288.76	Gloss: 0.23, 	Dloss: 66.45
Episode 11	Total Average Score: -1303.69	Gloss: 0.27, 	Dloss: 65.84
Episode 12	Total Average Score: -1318.03	Gloss: 0.33, 	Dloss: 64.99
Episode 13	Total Average Score: -1340.51	Gloss: 0.38, 	Dloss: 64.52
Episode 14	Total Average Score: -1381.06	Gloss: 0.44, 	Dloss: 65.73
Episode 15	Total Average Score: -1390.55	Gloss: 0.50, 	Dloss: 6

Episode 122	Total Average Score: -1641.36	Gloss: 7.27, 	Dloss: 66.11
Episode 123	Total Average Score: -1643.36	Gloss: 7.33, 	Dloss: 66.02
Episode 124	Total Average Score: -1644.42	Gloss: 7.40, 	Dloss: 65.90
Episode 125	Total Average Score: -1641.26	Gloss: 7.46, 	Dloss: 65.88
Episode 126	Total Average Score: -1638.29	Gloss: 7.52, 	Dloss: 65.79
Episode 127	Total Average Score: -1635.44	Gloss: 7.58, 	Dloss: 65.67
Episode 128	Total Average Score: -1634.20	Gloss: 7.64, 	Dloss: 65.71
Episode 129	Total Average Score: -1630.86	Gloss: 7.70, 	Dloss: 65.51
Episode 130	Total Average Score: -1627.70	Gloss: 7.76, 	Dloss: 65.51
Episode 131	Total Average Score: -1627.25	Gloss: 7.82, 	Dloss: 65.40
Episode 132	Total Average Score: -1625.37	Gloss: 7.88, 	Dloss: 65.39
Episode 133	Total Average Score: -1624.56	Gloss: 7.94, 	Dloss: 65.27
Episode 134	Total Average Score: -1622.07	Gloss: 8.00, 	Dloss: 65.18
Episode 135	Total Average Score: -1619.80	Gloss: 8.06, 	Dloss: 65.18
Episode 136	Total Average Score: -

Episode 240	Total Average Score: -1408.86	Gloss: 14.48, 	Dloss: 57.62
Episode 241	Total Average Score: -1406.91	Gloss: 14.54, 	Dloss: 57.59
Episode 242	Total Average Score: -1405.72	Gloss: 14.60, 	Dloss: 57.46
Episode 243	Total Average Score: -1405.70	Gloss: 14.66, 	Dloss: 57.52
Episode 244	Total Average Score: -1404.48	Gloss: 14.72, 	Dloss: 57.41
Episode 245	Total Average Score: -1403.71	Gloss: 14.79, 	Dloss: 57.33
Episode 246	Total Average Score: -1402.22	Gloss: 14.85, 	Dloss: 57.46
Episode 247	Total Average Score: -1405.14	Gloss: 14.91, 	Dloss: 57.16
Episode 248	Total Average Score: -1403.83	Gloss: 14.97, 	Dloss: 57.28
Episode 249	Total Average Score: -1403.02	Gloss: 15.03, 	Dloss: 57.11
Episode 250	Total Average Score: -1402.43	Gloss: 15.09, 	Dloss: 57.13
Episode 251	Total Average Score: -1400.95	Gloss: 15.16, 	Dloss: 57.11
Episode 252	Total Average Score: -1400.06	Gloss: 15.22, 	Dloss: 56.90
Episode 253	Total Average Score: -1399.36	Gloss: 15.28, 	Dloss: 57.00
Episode 254	Total Av

Episode 358	Total Average Score: -1304.24	Gloss: 21.73, 	Dloss: 52.67
Episode 359	Total Average Score: -1303.84	Gloss: 21.79, 	Dloss: 52.41
Episode 360	Total Average Score: -1302.45	Gloss: 21.85, 	Dloss: 52.51
Episode 361	Total Average Score: -1301.03	Gloss: 21.91, 	Dloss: 52.44
Episode 362	Total Average Score: -1299.84	Gloss: 21.97, 	Dloss: 52.32
Episode 363	Total Average Score: -1298.84	Gloss: 22.03, 	Dloss: 52.29
Episode 364	Total Average Score: -1298.02	Gloss: 22.10, 	Dloss: 52.46
Episode 365	Total Average Score: -1300.01	Gloss: 22.16, 	Dloss: 52.17
Episode 366	Total Average Score: -1299.62	Gloss: 22.22, 	Dloss: 52.33
Episode 367	Total Average Score: -1297.77	Gloss: 22.28, 	Dloss: 52.37
Episode 368	Total Average Score: -1297.24	Gloss: 22.34, 	Dloss: 52.36
Episode 369	Total Average Score: -1298.86	Gloss: 22.40, 	Dloss: 52.29
Episode 370	Total Average Score: -1297.96	Gloss: 22.47, 	Dloss: 52.51
Episode 371	Total Average Score: -1296.87	Gloss: 22.53, 	Dloss: 52.03
Episode 372	Total Av

### 5. Plot the result

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

plt.plot(np.arange(1, len(meanRs)+1), meanRs)
plt.ylabel('total_average_scores')
plt.xlabel('Episode #')
plt.show()

### 6. Watch the agent running with saved weights

In [14]:
# Load the saved weights into Pytorch model
agent.g.load_state_dict(torch.load('g-pendulum.pth', map_location='cpu'))
agent.d.load_state_dict(torch.load('d-pendulum.pth', map_location='cpu'))

# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# S = env_info.vector_observations                  # get the current states (S) (for each agent)
s = env.reset() # s: state # get the current state (s) (for an agent)

#Rs = np.zeros(num_agents)                          # initialize the total scores (Rs) (for each agent)
R = 0 # R:total reward # initialize the total score (R) (for an agent)

while True:
    #A = agent.act(S)                        # select actions (A) from loaded model agents
    a = agent.act(np.reshape(s, [1, -1])) # a: [-1, +1]    # select action (a) from loaded model agent
    
    # env_info = env.step(A)[brain_name]           # send all actions (A) to tne environment (env)
    # S2 = env_info.vector_observations         # get next states (S2) (for each agent)
    # rewards = env_info.rewards                         # get rewards (for each agent)
    # dones = env_info.local_done                        # see if the episode is done/finished (terminal)
    s2, r, done, _ = env.step(np.reshape(a, [-1]))
    
    # Rs += env_info.rewards                         # update the total scores (Rs) (for each agent)
    # S = S2                               # roll over current states (S) to next states (S2)
    R += r # update the total score (R) (for an agent)
    s = s2 # roll over current state (s) to next state (s2)
    
    #if np.any(dones):                                  # exit loop if episode is done/finished
    if done: # exit loop if episode is done/finished (terminal)
        break
        
print('Average of total scores: {}'.format(R))

FileNotFoundError: [Errno 2] No such file or directory: 'g-pendulum.pth'

When finished, you can close the environment.

In [6]:
env.close()